# **Use relevant features for Raga Identification**

# Importing Libraries and downloading data

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'indian-music-raga:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F815421%2F2029267%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240204%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240204T215410Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D23344f7c139a5874108449b3927c0a011d98f42ffe7db137ee64223d2e8752b12f7be63cb12bcc4b32752857c819455fd5b4778365f34492e368dfd9714d2d1202cd176007fdccad4049848c717168a0102e140f7e0042ab40443f3672c94472d8754b2f4e933b20c01d5ce724a26dd494f6aaf58e40d51b1c85bcb0b3d1fb5fd3df5f912bf7d54c1c06a195177fc10d9e60fafc2b4bd3aeebd83df708d1eabceab309082ceb2a3e3f4f78b1b5188ef60f3515aa8f1f15daae91e72743f78bb0b971c4d3a25055167518f146dd207f239c6e3826a35fcd300bf0249b25ef0748ea60a00247e88a8212e4c23cfea7eba4fc980cd66f9df91a95671823b518aaad'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1000941901 bytes downloaded
Downloaded and uncompressed: indian-music-raga
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/indian-music-raga/asavari27.wav
/kaggle/input/indian-music-raga/yaman02.wav
/kaggle/input/indian-music-raga/Bhairavi03.wav
/kaggle/input/indian-music-raga/yaman24.wav
/kaggle/input/indian-music-raga/DKanada04.wav
/kaggle/input/indian-music-raga/yaman04.wav
/kaggle/input/indian-music-raga/malkauns05.wav
/kaggle/input/indian-music-raga/bhoopali21.wav
/kaggle/input/indian-music-raga/bageshree31.wav
/kaggle/input/indian-music-raga/darbari26.wav
/kaggle/input/indian-music-raga/yaman01.wav
/kaggle/input/indian-music-raga/bhairavi30.wav
/kaggle/input/indian-music-raga/bhairavi27.wav
/kaggle/input/indian-music-raga/sarang05.wav
/kaggle/input/indian-music-raga/bhoop05.wav
/kaggle/input/indian-music-raga/sarang01.wav
/kaggle/input/indian-music-raga/yaman03.wav
/kaggle/input/indian-music-raga/bhairavi31.wav
/kaggle/input/indian-music-raga/asavari05.wav
/kaggle/input/indian-music-raga/DKanada05.wav
/kaggle/input/indian-music-raga/malkauns02.wav
/kaggle/input/indian-music-raga/asavari

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
list=[]
audio_files=["/kaggle/input/indian-music-raga/malkauns04.wav","/kaggle/input/indian-music-raga/yaman02.wav", "/kaggle/input/indian-music-raga/malkauns26.wav", "/kaggle/input/indian-music-raga/asavari25.wav", "/kaggle/input/indian-music-raga/bageshree04.wav", "/kaggle/input/indian-music-raga/sarang04.wav", "/kaggle/input/indian-music-raga/sarang16.wav", "/kaggle/input/indian-music-raga/bageshree29.wav", "/kaggle/input/indian-music-raga/bhairavi29.wav", "/kaggle/input/indian-music-raga/bhoop03.wav", "/kaggle/input/indian-music-raga/asavari26.wav", "/kaggle/input/indian-music-raga/Bhairavi01.wav", "/kaggle/input/indian-music-raga/yaman01.wav", "/kaggle/input/indian-music-raga/bhairavi30.wav", "/kaggle/input/indian-music-raga/yaman03.wav", "/kaggle/input/indian-music-raga/bhoop02.wav", "/kaggle/input/indian-music-raga/yaman27.wav", "/kaggle/input/indian-music-raga/bhairavi27.wav", "/kaggle/input/indian-music-raga/sarang05.wav", "/kaggle/input/indian-music-raga/yaman24.wav", "/kaggle/input/indian-music-raga/bhoopali24.wav", "/kaggle/input/indian-music-raga/DKanada01.wav", "/kaggle/input/indian-music-raga/bhoop04.wav", "/kaggle/input/indian-music-raga/bageshree27.wav", "/kaggle/input/indian-music-raga/asavari29.wav", "/kaggle/input/indian-music-raga/asavari28.wav", "/kaggle/input/indian-music-raga/darbari30.wav", "/kaggle/input/indian-music-raga/malkauns03.wav", "/kaggle/input/indian-music-raga/asavari03.wav", "/kaggle/input/indian-music-raga/sarang20.wav", "/kaggle/input/indian-music-raga/sarang02.wav", "/kaggle/input/indian-music-raga/darbari27.wav", "/kaggle/input/indian-music-raga/bageshree28.wav", "/kaggle/input/indian-music-raga/malkauns27.wav", "/kaggle/input/indian-music-raga/bhoopali22.wav", "/kaggle/input/indian-music-raga/malkauns28.wav", "/kaggle/input/indian-music-raga/DKanada05.wav", "/kaggle/input/indian-music-raga/Bhairavi04.wav", "/kaggle/input/indian-music-raga/yaman05.wav", "/kaggle/input/indian-music-raga/yaman23.wav", "/kaggle/input/indian-music-raga/yaman25.wav", "/kaggle/input/indian-music-raga/asavari02.wav", "/kaggle/input/indian-music-raga/bhoop01.wav", "/kaggle/input/indian-music-raga/bhoopali25.wav", "/kaggle/input/indian-music-raga/malkauns02.wav", "/kaggle/input/indian-music-raga/bhairavi28.wav", "/kaggle/input/indian-music-raga/yaman04.wav", "/kaggle/input/indian-music-raga/bageshree31.wav", "/kaggle/input/indian-music-raga/malkauns01.wav", "/kaggle/input/indian-music-raga/bageshree25.wav", "/kaggle/input/indian-music-raga/sarang18.wav", "/kaggle/input/indian-music-raga/asavari01.wav", "/kaggle/input/indian-music-raga/asavari04.wav", "/kaggle/input/indian-music-raga/bhoopali21.wav", "/kaggle/input/indian-music-raga/DKanada04.wav", "/kaggle/input/indian-music-raga/Bhairavi03.wav", "/kaggle/input/indian-music-raga/bageshree24.wav", "/kaggle/input/indian-music-raga/bhairavi31.wav", "/kaggle/input/indian-music-raga/malkauns29.wav", "/kaggle/input/indian-music-raga/bageshree05.wav", "/kaggle/input/indian-music-raga/bageshree01.wav", "/kaggle/input/indian-music-raga/bhoop05.wav", "/kaggle/input/indian-music-raga/bageshree30.wav", "/kaggle/input/indian-music-raga/darbari28.wav", "/kaggle/input/indian-music-raga/asavari05.wav", "/kaggle/input/indian-music-raga/sarang03.wav", "/kaggle/input/indian-music-raga/asavari27.wav", "/kaggle/input/indian-music-raga/malkauns25.wav", "/kaggle/input/indian-music-raga/Bhairavi05.wav", "/kaggle/input/indian-music-raga/sarang01.wav", "/kaggle/input/indian-music-raga/bhoopali23.wav", "/kaggle/input/indian-music-raga/sarang19.wav", "/kaggle/input/indian-music-raga/darbari26.wav", "/kaggle/input/indian-music-raga/Bhairavi02.wav", "/kaggle/input/indian-music-raga/malkauns05.wav", "/kaggle/input/indian-music-raga/bageshree02.wav", "/kaggle/input/indian-music-raga/sarang17.wav", "/kaggle/input/indian-music-raga/bageshree03.wav", "/kaggle/input/indian-music-raga/DKanada02.wav", "/kaggle/input/indian-music-raga/DKanada03.wav", "/kaggle/input/indian-music-raga/yaman26.wav", "/kaggle/input/indian-music-raga/darbari29.wav"]
list=le.fit_transform(audio_files)

In [ ]:
import cv2
import audioread
import logging
import os
import random
import time
import warnings
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
from contextlib import contextmanager
from pathlib import Path
from typing import Optional
from fastprogress import progress_bar
from sklearn.metrics import f1_score
from torchvision import models

In [ ]:
unique_names=["DKanada", "Asavari", "Bagehsree", "Bhairavi", "Bhoop", "Bhoopali", "Darbari", "Alkauns", "Sarang", "Yaman"]

In [ ]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(unique_names)

In [ ]:
RAGA_CODE= dict(zip(unique_names,encoded_labels))

In [ ]:
INV_RAGA_CODE={v: k for k, v in RAGA_CODE.items()}

# Data Processing

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [ ]:
def features_extractor(file):
  audio, sample_rate = librosa.load(file)
  mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate,n_mfcc= 40);
  mfccs_scaled_features = np.mean(mfccs.T, axis=0)

  return mfccs_scaled_features

In [ ]:
features_list=[]
for i in audio_files:
    label=(str(i[32:-6])).title()
    features_list.append([features_extractor(i),label])

In [ ]:
features_list

[[array([-2.0158595e+02,  1.5732413e+02, -5.0954742e+01, -1.3381189e+00,
         -5.2214656e+00,  2.5660619e-02, -3.2708450e+01, -9.7979002e+00,
         -1.1511979e+01, -4.1105757e+00,  5.1720381e-01, -7.0761361e+00,
          3.4661005e+00,  6.6538920e+00,  3.4325621e+00,  8.0415831e+00,
          4.9906049e+00,  3.3219998e+00, -3.8186667e+00, -4.7322288e+00,
         -5.0301986e+00, -1.4267447e+00, -1.1028236e+00, -8.1978855e+00,
          1.2108908e+00, -1.2918089e-01, -1.6260991e+00,  6.3336554e+00,
          3.2547584e+00, -5.0702224e+00,  3.5343361e-01,  7.1537533e+00,
         -8.1036747e-01, -1.6583667e+00,  3.2938311e-01, -3.8622882e+00,
         -3.3482179e-02, -1.3010518e-01, -1.6659706e+00,  6.3095016e+00],
        dtype=float32),
  'Malkauns'],
 [array([-2.35489426e+02,  1.87521408e+02, -5.87848816e+01, -2.58163071e+01,
          1.21863079e+01, -6.51940012e+00, -2.24739475e+01, -8.36923027e+00,
         -7.53815937e+00,  3.58125281e+00,  2.48213005e+00, -3.22271395e+00,

In [ ]:
features_df=pd.DataFrame(features_list,columns=['feature','class'])
features_df.head()

,feature,class
0,"[-201.58595, 157.32413, -50.954742, -1.3381189...",Malkauns
1,"[-235.48943, 187.52141, -58.78488, -25.816307,...",Yaman
2,"[-340.8554, 73.094574, -5.8470564, 28.104563, ...",Malkauns
3,"[-297.42203, 58.901947, -9.98139, 33.196007, -...",Asavari
4,"[-213.21152, 163.44122, -45.60164, -18.607985,...",Bageshree


In [ ]:
X=np.array(features_df['feature'].tolist())
y=np.array(features_df['class'].tolist())
le=LabelEncoder()
ti=le.fit_transform(y)
y=to_categorical(ti)

In [ ]:
originallabels=le.classes_
print("Original labels: ", originallabels)
print("Encoded labels: ", np.unique(ti))

Original labels:  ['Asavari' 'Bageshree' 'Bhairavi' 'Bhoop' 'Bhoopali' 'Darbari' 'Dkanada'
 'Malkauns' 'Sarang' 'Yaman']
Encoded labels:  [0 1 2 3 4 5 6 7 8 9]


# Training Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=48)

In [ ]:
print(f"{y_test.shape}")
print(f"{y_train.shape}")

(17, 10)
(65, 10)


In [ ]:
import tensorflow as tf

In [ ]:
X_train_flat = np.array([x.flatten() for x in X_train])
X_test_flat = np.array([x.flatten() for x in X_test])

In [ ]:
print(X_train_flat)

[[-2.3548943e+02  1.8752141e+02 -5.8784882e+01 ... -3.3659134e-02
   5.6613889e+00  4.0233626e+00]
 [-3.0949637e+02  6.9884613e+01 -6.3524208e+00 ...  6.7624574e+00
   9.1562405e-02 -2.0142133e+00]
 [-2.7996729e+02  5.2106136e+01 -1.0332816e+01 ...  5.0621991e+00
  -1.9010776e+00 -9.1663104e-01]
 ...
 [-1.0985678e+02  1.5102933e+02 -3.8895462e+01 ... -3.2200754e+00
  -3.4730299e+00 -1.2488182e+00]
 [-9.2346550e+01  1.6161327e+02 -7.3482483e+01 ... -4.1735225e+00
   4.0908904e+00  6.1521044e+00]
 [-2.0158595e+02  1.5732413e+02 -5.0954742e+01 ... -1.3010518e-01
  -1.6659706e+00  6.3095016e+00]]


In [ ]:
from tensorflow.keras import layers, models

In [ ]:
model3 = models.Sequential()
model3.add(layers.Conv1D(32, kernel_size=5, activation='relu', input_shape=(X_train_flat.shape[1], 1)))
model3.add(layers.MaxPooling1D(pool_size=1))
model3.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
model3.add(layers.MaxPooling1D(pool_size=1))
model3.add(layers.Flatten())
model3.add(layers.Dense(128, activation='relu'))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(10, activation='softmax'))
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model3.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
X_train_reshaped = X_train_flat.reshape((X_train_flat.shape[0], X_train_flat.shape[1], 1))
X_test_reshaped = X_test_flat.reshape((X_test_flat.shape[0], X_test_flat.shape[1], 1))

In [ ]:
len(X_train_reshaped)
len(y_train)


65

In [ ]:
model3.fit(X_train_reshaped, y_train, epochs=35, batch_size=4, validation_data=(X_test_reshaped, y_test))

Epoch 1/35
17/17 [==============================] - 2s 27ms/step - loss: 3.8927 - accuracy: 0.0923 - val_loss: 1.8909 - val_accuracy: 0.2353
Epoch 2/35
17/17 [==============================] - 0s 11ms/step - loss: 1.9867 - accuracy: 0.2615 - val_loss: 1.9532 - val_accuracy: 0.4118
Epoch 3/35
17/17 [==============================] - 0s 9ms/step - loss: 1.6190 - accuracy: 0.4462 - val_loss: 1.4814 - val_accuracy: 0.5294
Epoch 4/35
17/17 [==============================] - 0s 8ms/step - loss: 1.3337 - accuracy: 0.6154 - val_loss: 1.2369 - val_accuracy: 0.4706
Epoch 5/35
17/17 [==============================] - 0s 9ms/step - loss: 0.9904 - accuracy: 0.6769 - val_loss: 1.0495 - val_accuracy: 0.5882
Epoch 6/35
17/17 [==============================] - 0s 8ms/step - loss: 0.9649 - accuracy: 0.6769 - val_loss: 1.1104 - val_accuracy: 0.6471
Epoch 7/35
17/17 [==============================] - 0s 7ms/step - loss: 0.8649 - accuracy: 0.7077 - val_loss: 0.9785 - val_accuracy: 0.7059
Epoch 8/35
17/17 [

In [ ]:
accuracy = model3.evaluate(X_test_reshaped, y_test)[1]
print(f"Accuracy on the test set: {accuracy*100:.2f}%")

1/1 [==============================] - 0s 30ms/step - loss: 0.2836 - accuracy: 0.8235
Accuracy on the test set: 82.35%
